# Text2Bool

It would be useful for us (from a compliance verification point-of-view) if we could have a text2bool tool that could take in a sentence from the AI act and give out a boolean expression in multiple boolean variables that's output would be a boolean value representing whether the user is compliant with that particular regulation.

We found that simple relations (e.g. edges in a knowledge graph) struggle to encode some of the more complicated, conditional requirements seen in the AI Act. 
Boolean expressions could be used to encode the entire rulesego

e.g. A simplified ruleset could be

`compliant = (used_large_compute && is_open_source) || (using_facial_recognition && (for_research_purposes  && is_open_source) || (used_by_governments && assists_in_criminal_investigations))`

This reflects the fact that certain techniques are only allowed in certain circumstances (e.g. facial recognition may be allowed only if it's open source and for research purposes or used by the government for crime investigation).

## Generating rulesets

So far, we pass in legal texts sentence-by-sentence and try to extract relationships from them, e.g. "AI systems cannot use facial recognition". If we picked up on a relationship like that then we would use that in the knowledge graph, which would be applied to all AI systems. This is obviously a naive approach as there are certain situations where this doesn't apply.

We now have access to metadata about each sentence, such as the article number and point/subpoint it came from. This is useful because if we know that all of the rules in a certain article only apply in a certain situation, then we can append that to the boolean ruleset. E.g. if we read "Models under 1 billion parameters are not subject to the regulations set out in Article X", then we can find that section of the ruleset and append `&& has_over_1B_params` to that section of the ruleset. 

We can probably implement this as a custom Python class that has some functionality built on top of a standard boolean variable

In [10]:
class ExtendedBool:
    def __init__(self, value, case_or_situation=None, article_num=0):
        self.value = value
        self.case_or_situation = case_or_situation
        self.article_num = article_num

    # TODO: Overload the logical opeerators ("and", "or", etc) instead of these functions

    def _and(self, other):
        return ExtendedBool(self.value and other.value)

    def _or(self, other):
        return ExtendedBool(self.value or other.value)

    def __repr__(self):
        return f"{self.value}"

    
bool1 = ExtendedBool(True, 24)
bool2 = ExtendedBool(False, 12)
bool3 = bool1._and(bool2)
bool4 = bool1._or(bool2)
print(bool3, bool4)


False True


In [15]:
class Ruleset:
    def __init__(self, ruleset):
        self.ruleset = ruleset
    
    # This might not work because it could evaluate the ruleset to true or false before, 
    #   so I need to think of a way to represent a boolean expression rather than just a boolean variable
    def modify(self, article_num, case_or_situation, point_num=None, subpoint_num=None):
        # TODO: Add functionality for points and subpoints
        if not point_num and not subpoint_num:
            self.ruleset[article_num] = self.ruleset[article_num] and case_or_situation

In [16]:
# User model data, pretend this was extracted from the model documentation
is_over_1b_params = False
is_open_source = True
for_research_purposes = True

ruleset_dict = {
    "Article 1":
    {
        "Point 1": is_open_source or for_research_purposes
    }
}

ruleset = Ruleset(ruleset_dict)
ruleset.modify("Article 1", ExtendedBool(True, is_over_1b_params))

In [17]:
# TODO: Finish the following example code to "automatically" change the ruleset given a sentence.

def find_which_rules_apply_to_which_articles(ai_act_sentence, ruleset):
    # Pretend there's some actual code here... Maybe we can get an LLM to do this on the fly
    if sentence == "Article 1 does not apply to models under 1b params":
        ruleset.modify("Article 1", ExtendedBool(True, is_over_1b_params))
    elif sentence == "Article 2 does not apply to open_source_models":
        ruleset.modify("Article 2", ExtendedBool(True, is_open_source))


sentence = "Article 1 does not apply to models under 1b params"
find_which_rules_apply_to_which_articles(sentence, ruleset)


# How to represent Boolean Expressions
We can have an extended Boolean Expression class that contains one or two boolean operands and a boolean operator. We can construct Boolean Expressions from these and evaluate them by recursively evaluating the sub-boolean expressions.


In [ ]:
class ExtendedBooleanExpression:
    def __init__(self, operator, first, second=None):
        self.operator = operator
        self.first = first
        self.second = second
        self.evaluate()

    def evaluate(self):
        # TODO: Make the below line work, basically evaluate the boolean expression
        # self.value = self.first self.operator self.second
    
    